In [2]:
# Veri Temizleme
import pandas as pd
df = pd.read_csv("S&P 500 Historical.csv")
df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'], format='%d/%m/%Y')  
df["Date"] = df['Unnamed: 0']
df_cleaned = df.dropna(subset=['SP500 status of AAPL'])

# Şirketlerin İsimlerini Toplama

# Kontol etmeden direkt kolon sayısından alırsak şirket sayısı 1175
status_cols = [col for col in df.columns if col.startswith("SP500 status of")]
ticker_list = [col.replace("SP500 status of ", "") for col in status_cols]
print(len(ticker_list))

# Hiç aktif olmuş mu diye kontol ederek alırsak şirket sayısı 1060
status_cols = [col for col in df.columns if col.startswith("SP500 status of")]
df_status = df[status_cols]
membership_bool = df_status.any(axis=0)
num_companies = membership_bool.sum()
print(f"2000'den günümüze S&P 500'de yer almış toplam farklı şirket sayısı: {num_companies}")
tickers_in_sp500 = [
    col.replace("SP500 status of ", "") 
    for col in membership_bool[membership_bool].index
]

# Aradaki farkın doğruluğunun testi
fark_liste1 = set(tickers_in_sp500) - set(ticker_list)
fark_liste2 = set(ticker_list) - set(tickers_in_sp500)
simetrik_fark = set(tickers_in_sp500).symmetric_difference(set(ticker_list))
for ticker in simetrik_fark:
    col_name = f"SP500 status of {ticker}"
    if col_name in df.columns:
        if df[col_name].eq(1).any():
            print(f"{col_name}: sütununda en az bir '1' değeri var.")
        else:
            pass
    else:
        print(f"{col_name} sütunu DataFrame'de mevcut değil.")


1175
2000'den günümüze S&P 500'de yer almış toplam farklı şirket sayısı: 1060


In [1]:
# Şirketlerin aktif oldukları dönemleri
active_periods = {}
for ticker in tickers_in_sp500:
    col_name = f"SP500 status of {ticker}"
    if col_name in df.columns:
        active_dates = df.loc[df[col_name] == 1, 'date']
        active_periods[ticker] = active_dates.tolist()
    else:
        active_periods[ticker] = []
for ticker, dates in active_periods.items():
    print(f"{ticker} aktif olduğu zamanlar: {dates}")

NameError: name 'tickers_in_sp500' is not defined

In [ ]:
# Çeyreklere ayırarak o çeyrekten itibaren olan şirketler (45 gün eklenmeden hesaplandı)
df["date"] = pd.to_datetime(df["date"], dayfirst=True)
df["quarter"] = df["date"].dt.to_period("Q")
ticker_cols = [col for col in df.columns if col.startswith("SP500 status of ")]
quarters = sorted(df["quarter"].unique())
results_q = {}
for q in quarters:
    start_date = q.start_time
    subset = df[df["date"] >= start_date]
    active_tickers = []
    for col in ticker_cols:
        if subset[col].eq(1).any():
            ticker_name = col.replace("SP500 status of ", "")
            active_tickers.append(ticker_name)
        results_q[str(q)] = active_tickers
for quarter, tickers in results_q.items():
    print(f"{quarter} itibarıyla aktif şirketler ({len(tickers)} adet): {tickers}")

2000Q1 itibarıyla aktif şirketler (1060 adet): ['LKQ', 'SHW', 'AWK', 'BX', 'PALM', 'APOL', 'DHR', 'UPC', 'EOP', 'HPQ', 'PDCO', 'XOM', 'CITGQ', 'FDS', 'PARA', 'OXY', 'SCI', 'M', 'USB', 'SNA', 'THC', 'KSE', 'PPL', 'ADI', 'JOY', 'GAS', 'GX', 'WBA', 'BOL', 'ASH', 'GP', 'SNI', 'VZ', 'AAP', 'AOS', 'WPX', 'TDC', 'SNV', 'PDG', 'HRL', 'URBN', 'GOOGL', 'LO', 'ESV', 'EXC', 'RJF', 'HON', 'UTX', 'WRB', 'WLL', 'SJM', 'TER', 'FSL', 'NOVL', 'CDW', 'MAR', 'RAL', 'MMI', 'BALL', 'ANTM', 'MNK', 'CTXS', 'MBI', 'AABA', 'PRU', 'ARNC', 'UN', 'HRS', 'HD', 'OAT', 'HBAN', 'MYL', 'NGH', 'FLT', 'UAA', 'ALL', 'RHI', 'SII', 'WRK', 'WAB', 'MWW', 'OGN', 'DXCM', 'BEAM', 'NMK', 'ETFC', 'GENZ', 'PHA', 'AT', 'SWN', 'DVA', 'GTW', 'EIX', 'WAMUQ', 'CF', 'VIAV', 'MAS', 'H', 'HRB', 'FII', 'IBM', 'CBRE', 'CFN', 'UNM', 'BMY', 'SOV', 'MLM', 'MSCI', 'SPLS', 'NLOK', 'SBNY', 'MCK', 'JNJ', 'WST', 'AMCR', 'NEM', 'BLK', 'LUMN', 'TRGP', 'KSS', 'ACKH', 'AEP', 'STX', 'CMX', 'KMI', 'BG', 'GTE', 'UST', 'DFS', 'HCP', 'PWJ', 'FSH', 'VRTX', 'U

In [ ]:
# Çeyreklere ayırarak o çeyrekten itibaren olan şirketler (45 gün eklenerek hesaplandı)
df["date"] = pd.to_datetime(df["date"], dayfirst=True)
df["quarter"] = df["date"].dt.to_period("Q")
ticker_cols = [col for col in df.columns if col.startswith("SP500 status of ")]
quarters = sorted(df["quarter"].unique())
results_q = {}
for q in quarters:
    start_date = q.start_time + pd.Timedelta(days=45)
    subset = df[df["date"] >= start_date]
    active_tickers = []
    for col in ticker_cols:
        if subset[col].eq(1).any():
            ticker_name = col.replace("SP500 status of ", "")
            active_tickers.append(ticker_name)
    results_q[str(q)] = active_tickers
for quarter, tickers in results_q.items():
    print(f"{quarter} itibarıyla aktif şirketler ({len(tickers)} adet): {tickers}")

2000Q1 itibarıyla aktif şirketler (1056 adet): ['LKQ', 'SHW', 'AWK', 'BX', 'PALM', 'APOL', 'DHR', 'UPC', 'EOP', 'HPQ', 'PDCO', 'XOM', 'CITGQ', 'FDS', 'PARA', 'OXY', 'SCI', 'M', 'USB', 'SNA', 'THC', 'KSE', 'PPL', 'ADI', 'JOY', 'GAS', 'GX', 'WBA', 'BOL', 'ASH', 'GP', 'SNI', 'VZ', 'AAP', 'AOS', 'WPX', 'TDC', 'SNV', 'PDG', 'HRL', 'URBN', 'GOOGL', 'LO', 'ESV', 'EXC', 'RJF', 'HON', 'UTX', 'WRB', 'WLL', 'SJM', 'TER', 'FSL', 'NOVL', 'CDW', 'MAR', 'RAL', 'MMI', 'BALL', 'ANTM', 'MNK', 'CTXS', 'MBI', 'AABA', 'PRU', 'ARNC', 'UN', 'HRS', 'HD', 'OAT', 'HBAN', 'MYL', 'NGH', 'FLT', 'UAA', 'ALL', 'RHI', 'SII', 'WRK', 'WAB', 'MWW', 'OGN', 'DXCM', 'BEAM', 'NMK', 'ETFC', 'GENZ', 'PHA', 'AT', 'SWN', 'DVA', 'GTW', 'EIX', 'WAMUQ', 'CF', 'VIAV', 'MAS', 'H', 'HRB', 'FII', 'IBM', 'CBRE', 'CFN', 'UNM', 'BMY', 'SOV', 'MLM', 'MSCI', 'SPLS', 'NLOK', 'SBNY', 'MCK', 'JNJ', 'WST', 'AMCR', 'NEM', 'BLK', 'LUMN', 'TRGP', 'KSS', 'ACKH', 'AEP', 'STX', 'CMX', 'KMI', 'BG', 'GTE', 'UST', 'DFS', 'HCP', 'PWJ', 'FSH', 'VRTX', 'U

In [ ]:
# Yıllara ayırarak o yıldan itibaren olan şirketler (90 gün eklenmeden hesaplanmıştır)
df["date"] = pd.to_datetime(df["date"], dayfirst=True)
df["year"] = df["date"].dt.to_period("Y")
ticker_cols = [col for col in df.columns if col.startswith("SP500 status of ")]
years = sorted(df["year"].unique())
results_year = {}
for y in years:
    start_date = y.start_time
    subset_year = df[df["date"] >= start_date]
    active_tickers = []
    for col in ticker_cols:
        if subset_year[col].eq(1).any():
            ticker_name = col.replace("SP500 status of ", "")
            active_tickers.append(ticker_name)
    results_year[str(y)] = active_tickers
for year, tickers in results_year.items():
    print(f"{year} itibarıyla aktif şirketler ({len(tickers)} adet): {tickers}")

2000 itibarıyla aktif şirketler (1060 adet): ['LKQ', 'SHW', 'AWK', 'BX', 'PALM', 'APOL', 'DHR', 'UPC', 'EOP', 'HPQ', 'PDCO', 'XOM', 'CITGQ', 'FDS', 'PARA', 'OXY', 'SCI', 'M', 'USB', 'SNA', 'THC', 'KSE', 'PPL', 'ADI', 'JOY', 'GAS', 'GX', 'WBA', 'BOL', 'ASH', 'GP', 'SNI', 'VZ', 'AAP', 'AOS', 'WPX', 'TDC', 'SNV', 'PDG', 'HRL', 'URBN', 'GOOGL', 'LO', 'ESV', 'EXC', 'RJF', 'HON', 'UTX', 'WRB', 'WLL', 'SJM', 'TER', 'FSL', 'NOVL', 'CDW', 'MAR', 'RAL', 'MMI', 'BALL', 'ANTM', 'MNK', 'CTXS', 'MBI', 'AABA', 'PRU', 'ARNC', 'UN', 'HRS', 'HD', 'OAT', 'HBAN', 'MYL', 'NGH', 'FLT', 'UAA', 'ALL', 'RHI', 'SII', 'WRK', 'WAB', 'MWW', 'OGN', 'DXCM', 'BEAM', 'NMK', 'ETFC', 'GENZ', 'PHA', 'AT', 'SWN', 'DVA', 'GTW', 'EIX', 'WAMUQ', 'CF', 'VIAV', 'MAS', 'H', 'HRB', 'FII', 'IBM', 'CBRE', 'CFN', 'UNM', 'BMY', 'SOV', 'MLM', 'MSCI', 'SPLS', 'NLOK', 'SBNY', 'MCK', 'JNJ', 'WST', 'AMCR', 'NEM', 'BLK', 'LUMN', 'TRGP', 'KSS', 'ACKH', 'AEP', 'STX', 'CMX', 'KMI', 'BG', 'GTE', 'UST', 'DFS', 'HCP', 'PWJ', 'FSH', 'VRTX', 'UHS

In [ ]:
# Yıllara ayırarak o yıldan itibaren olan şirketler (90 gün eklenerek hesaplandı)
df["date"] = pd.to_datetime(df["date"], dayfirst=True)
df["year"] = df["date"].dt.to_period("Y")
ticker_cols = [col for col in df.columns if col.startswith("SP500 status of ")]
years = sorted(df["year"].unique())
results_year = {}
for y in years:
    start_date = y.start_time + pd.Timedelta(days=90)
    subset_year = df[df["date"] >= start_date]
    active_tickers = []
    for col in ticker_cols:
        if subset_year[col].eq(1).any():
            ticker_name = col.replace("SP500 status of ", "")
            active_tickers.append(ticker_name)
    results_year[str(y)] = active_tickers
for year, tickers in results_year.items():
    print(f"{year} itibarıyla aktif şirketler ({len(tickers)} adet): {tickers}")

2000 itibarıyla aktif şirketler (1053 adet): ['LKQ', 'SHW', 'AWK', 'BX', 'PALM', 'APOL', 'DHR', 'UPC', 'EOP', 'HPQ', 'PDCO', 'XOM', 'CITGQ', 'FDS', 'PARA', 'OXY', 'M', 'USB', 'SNA', 'THC', 'KSE', 'PPL', 'ADI', 'JOY', 'GAS', 'GX', 'WBA', 'BOL', 'ASH', 'GP', 'SNI', 'VZ', 'AAP', 'AOS', 'WPX', 'TDC', 'SNV', 'PDG', 'HRL', 'URBN', 'GOOGL', 'LO', 'ESV', 'EXC', 'RJF', 'HON', 'UTX', 'WRB', 'WLL', 'SJM', 'TER', 'FSL', 'NOVL', 'CDW', 'MAR', 'RAL', 'MMI', 'BALL', 'ANTM', 'MNK', 'CTXS', 'MBI', 'AABA', 'PRU', 'ARNC', 'UN', 'HRS', 'HD', 'OAT', 'HBAN', 'MYL', 'NGH', 'FLT', 'UAA', 'ALL', 'RHI', 'SII', 'WRK', 'WAB', 'MWW', 'OGN', 'DXCM', 'BEAM', 'NMK', 'ETFC', 'GENZ', 'PHA', 'AT', 'SWN', 'DVA', 'GTW', 'EIX', 'WAMUQ', 'CF', 'VIAV', 'MAS', 'H', 'HRB', 'FII', 'IBM', 'CBRE', 'CFN', 'UNM', 'BMY', 'SOV', 'MLM', 'MSCI', 'SPLS', 'NLOK', 'SBNY', 'MCK', 'JNJ', 'WST', 'AMCR', 'NEM', 'BLK', 'LUMN', 'TRGP', 'KSS', 'ACKH', 'AEP', 'STX', 'CMX', 'KMI', 'BG', 'GTE', 'UST', 'DFS', 'HCP', 'PWJ', 'FSH', 'VRTX', 'UHS', 'MAC

In [ ]:
# Çeyreklere ayırarak o çeyrekte aktif olan şirketler (45 gün eklenmeden hesaplandı)
df["date"] = pd.to_datetime(df["date"], dayfirst=True)
df["quarter"] = df["date"].dt.to_period("Q")
ticker_cols = [col for col in df.columns if col.startswith("SP500 status of ")]
quarters = sorted(df["quarter"].unique())
results_q = {}
for q in quarters:
    subset = df[df["quarter"] == q]
    active_tickers = []
    for col in ticker_cols:
        if subset[col].eq(1).any():
            ticker_name = col.replace("SP500 status of ", "")
            active_tickers.append(ticker_name)
    results_q[str(q)] = active_tickers
for quarter, tickers in results_q.items():
    print(f"{quarter} çeyreğinde aktif şirketler ({len(tickers)} adet): {tickers}")

2000Q1 çeyreğinde aktif şirketler (495 adet): ['SHW', 'DHR', 'UPC', 'HPQ', 'XOM', 'OXY', 'SCI', 'M', 'USB', 'SNA', 'THC', 'PPL', 'ADI', 'GAS', 'GX', 'WBA', 'BOL', 'ASH', 'GP', 'VZ', 'SNV', 'PDG', 'EXC', 'HON', 'UTX', 'WLL', 'TER', 'NOVL', 'MAR', 'RAL', 'CTXS', 'MBI', 'ARNC', 'UN', 'HD', 'OAT', 'HBAN', 'NGH', 'ALL', 'BEAM', 'NMK', 'PHA', 'AT', 'GTW', 'EIX', 'WAMUQ', 'MAS', 'H', 'HRB', 'IBM', 'UNM', 'BMY', 'SPLS', 'MCK', 'JNJ', 'NEM', 'KSS', 'ACKH', 'AEP', 'GTE', 'UST', 'PWJ', 'ETN', 'CNG', 'HNZ', 'PPG', 'BCR', 'HET', 'RDS.A', 'COMS', 'COP', 'D', 'GPU', 'AFS.A', 'PH', 'HPC', 'CTB', 'DXC', 'AGN', 'EKDKQ', 'HCR', 'CAT', 'ONE', 'BSX', 'APA', 'SEE', 'BHGE', 'DOV', 'LB', 'NWL', 'NYT', 'HAS', 'K', 'CMCSA', 'KR', 'DJ', 'DD', 'PCH', 'AFL', 'F', 'MEL', 'OWENQ', 'NRTLQ', 'STJ', 'PCS', 'AXP', 'FJ', 'BEN', 'WCOEQ', 'CMVT', 'EMC', 'SXCL', 'PHM', 'CR', 'CGP', 'PCAR', 'AM', 'SAF', 'WMT', 'CSCO', 'GPC', 'LSI', 'APC', 'GRA', 'DALRQ', 'ABI', 'SLB', 'CTX', 'SMI', 'BR', 'TOS', 'KO', 'DIS', 'JCI', 'MCO', 'AM

In [12]:
# Çeyreklere ayırarak o çeyrekte aktif olan şirketler (45 gün eklenmeden hesaplandı)
df["date"] = pd.to_datetime(df["date"], dayfirst=True)
df["quarter"] = df["date"].dt.to_period("Q")
ticker_cols = [col for col in df.columns if col.startswith("SP500 status of ")]
quarters = sorted(df["quarter"].unique())
results_q = {}
for q in quarters:
    quarter_start = q.start_time
    quarter_end = q.end_time
    extended_end = quarter_end + pd.Timedelta(days=45)
    subset = df[(df["date"] >= quarter_start) & (df["date"] <= extended_end)]    
    active_tickers = []
    for col in ticker_cols:
        if subset[col].eq(1).any():
            ticker_name = col.replace("SP500 status of ", "")
            active_tickers.append(ticker_name)
    results_q[str(q)] = active_tickers
for quarter, tickers in results_q.items():
    print(f"{quarter} çeyreğinde aktif şirketler ({len(tickers)} adet): {tickers}")

2000Q1 çeyreğinde aktif şirketler (501 adet): ['SHW', 'DHR', 'UPC', 'HPQ', 'XOM', 'OXY', 'SCI', 'M', 'USB', 'SNA', 'THC', 'PPL', 'ADI', 'GAS', 'GX', 'WBA', 'BOL', 'ASH', 'GP', 'VZ', 'SNV', 'PDG', 'EXC', 'HON', 'UTX', 'WLL', 'TER', 'NOVL', 'MAR', 'RAL', 'CTXS', 'MBI', 'ARNC', 'UN', 'HD', 'OAT', 'HBAN', 'NGH', 'ALL', 'BEAM', 'NMK', 'PHA', 'AT', 'GTW', 'EIX', 'WAMUQ', 'MAS', 'H', 'HRB', 'IBM', 'UNM', 'BMY', 'SPLS', 'MCK', 'JNJ', 'NEM', 'KSS', 'ACKH', 'AEP', 'GTE', 'UST', 'PWJ', 'ETN', 'CNG', 'HNZ', 'PPG', 'BCR', 'HET', 'RDS.A', 'COMS', 'COP', 'D', 'GPU', 'AFS.A', 'PH', 'HPC', 'CTB', 'DXC', 'AGN', 'EKDKQ', 'HCR', 'CAT', 'ONE', 'BSX', 'APA', 'SEE', 'BHGE', 'DOV', 'LB', 'NWL', 'NYT', 'HAS', 'K', 'CMCSA', 'KR', 'DJ', 'DD', 'PCH', 'AFL', 'F', 'MEL', 'OWENQ', 'NRTLQ', 'STJ', 'PCS', 'AXP', 'FJ', 'BEN', 'WCOEQ', 'CMVT', 'EMC', 'SXCL', 'PHM', 'CR', 'CGP', 'PCAR', 'AM', 'SAF', 'WMT', 'CSCO', 'GPC', 'LSI', 'APC', 'GRA', 'DALRQ', 'ABI', 'SLB', 'CTX', 'SMI', 'BR', 'TOS', 'KO', 'DIS', 'JCI', 'MCO', 'AM

In [ ]:
# Yıllara ayırarak o yılda aktif olan şirketler (90 gün eklenmeden hesaplandı)
df["date"] = pd.to_datetime(df["date"], dayfirst=True)
df["year"] = df["date"].dt.to_period("Y")
ticker_cols = [col for col in df.columns if col.startswith("SP500 status of ")]
years = sorted(df["year"].unique())
results_year = {}
for y in years:
    start_date = y.start_time
    end_date = y.end_time
    subset_year = df[(df["date"] >= start_date) & (df["date"] <= end_date)]    
    active_tickers = []
    for col in ticker_cols:
        if subset_year[col].eq(1).any():
            ticker_name = col.replace("SP500 status of ", "")
            active_tickers.append(ticker_name)
    results_year[str(y)] = active_tickers
for year, tickers in results_year.items():
    print(f"{year} yılı içerisinde aktif şirketler ({len(tickers)} adet): {tickers}")

2000 yılı içerisinde aktif şirketler (543 adet): ['SHW', 'PALM', 'DHR', 'UPC', 'HPQ', 'XOM', 'OXY', 'SCI', 'M', 'USB', 'SNA', 'THC', 'KSE', 'PPL', 'ADI', 'GAS', 'GX', 'WBA', 'BOL', 'ASH', 'GP', 'VZ', 'SNV', 'PDG', 'EXC', 'HON', 'UTX', 'WLL', 'TER', 'NOVL', 'MAR', 'RAL', 'CTXS', 'MBI', 'AABA', 'ARNC', 'UN', 'HD', 'OAT', 'HBAN', 'NGH', 'ALL', 'RHI', 'BEAM', 'NMK', 'PHA', 'AT', 'GTW', 'EIX', 'WAMUQ', 'CF', 'VIAV', 'MAS', 'H', 'HRB', 'IBM', 'UNM', 'BMY', 'SPLS', 'MCK', 'JNJ', 'NEM', 'KSS', 'ACKH', 'AEP', 'KMI', 'GTE', 'UST', 'PWJ', 'DYN', 'ETN', 'MERQ', 'CNG', 'HNZ', 'PPG', 'BCR', 'HET', 'RDS.A', 'COMS', 'COP', 'D', 'GPU', 'AFS.A', 'PH', 'HPC', 'CTB', 'DXC', 'AGN', 'EKDKQ', 'HCR', 'NVLS', 'CAT', 'SANM', 'ONE', 'BRCM', 'BSX', 'APA', 'SEE', 'BHGE', 'DOV', 'LB', 'NWL', 'NYT', 'HAS', 'K', 'CMCSA', 'KR', 'DJ', 'DD', 'PCH', 'AFL', 'F', 'MEL', 'OWENQ', 'NRTLQ', 'STJ', 'PCS', 'AXP', 'FJ', 'BEN', 'WCOEQ', 'CMVT', 'EMC', 'SXCL', 'PHM', 'CR', 'CGP', 'PCAR', 'AM', 'SAF', 'WMT', 'CSCO', 'GPC', 'LSI', '

In [ ]:
# Yıllara ayırarak o yılda aktif olan şirketler (90 gün eklenerek hesaplandı)
df["date"] = pd.to_datetime(df["date"], dayfirst=True)
df["year"] = df["date"].dt.to_period("Y")
ticker_cols = [col for col in df.columns if col.startswith("SP500 status of ")]
years = sorted(df["year"].unique())
results_year = {}
for y in years:
    start_date = y.start_time
    end_date = y.end_time
    extended_end = end_date + pd.Timedelta(days=90)
    subset_year = df[(df["date"] >= start_date) & (df["date"] <= extended_end)]
    active_tickers = []
    for col in ticker_cols:
        if subset_year[col].eq(1).any():
            ticker_name = col.replace("SP500 status of ", "")
            active_tickers.append(ticker_name)
    results_year[str(y)] = active_tickers
for year, tickers in results_year.items():
    print(f"{year} yılı içerisinde aktif şirketler ({len(tickers)} adet): {tickers}")

2000 yılı içerisinde aktif şirketler (549 adet): ['SHW', 'PALM', 'DHR', 'UPC', 'HPQ', 'XOM', 'OXY', 'SCI', 'M', 'USB', 'SNA', 'THC', 'KSE', 'PPL', 'ADI', 'GAS', 'GX', 'WBA', 'BOL', 'ASH', 'GP', 'VZ', 'SNV', 'PDG', 'EXC', 'HON', 'UTX', 'WLL', 'TER', 'NOVL', 'MAR', 'RAL', 'CTXS', 'MBI', 'AABA', 'ARNC', 'UN', 'HD', 'OAT', 'HBAN', 'NGH', 'ALL', 'RHI', 'BEAM', 'NMK', 'PHA', 'AT', 'GTW', 'EIX', 'WAMUQ', 'CF', 'VIAV', 'MAS', 'H', 'HRB', 'IBM', 'UNM', 'BMY', 'SPLS', 'MCK', 'JNJ', 'NEM', 'KSS', 'ACKH', 'AEP', 'KMI', 'GTE', 'UST', 'PWJ', 'DYN', 'ETN', 'MERQ', 'CNG', 'HNZ', 'PPG', 'BCR', 'HET', 'RDS.A', 'COMS', 'COP', 'D', 'GPU', 'AFS.A', 'PH', 'HPC', 'CTB', 'DXC', 'AGN', 'EKDKQ', 'HCR', 'NVLS', 'CAT', 'SANM', 'ONE', 'BRCM', 'BSX', 'APA', 'SEE', 'BHGE', 'DOV', 'LB', 'NWL', 'NYT', 'HAS', 'K', 'CMCSA', 'KR', 'DJ', 'DD', 'PCH', 'AFL', 'F', 'MEL', 'OWENQ', 'NRTLQ', 'STJ', 'PCS', 'AXP', 'FJ', 'BEN', 'WCOEQ', 'CMVT', 'EMC', 'SXCL', 'PHM', 'CR', 'CGP', 'PCAR', 'AM', 'SAF', 'WMT', 'CSCO', 'GPC', 'LSI', '